In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

DATAPATH = "F:/data/"

K = 5 # top-k的k值
TEST_SIZE = 0.99 # 测试集的占比

In [2]:
df = pd.read_csv(DATAPATH + "rating.csv")
print(df.shape)
df.head(5)

(58381, 3)


,userID,movieID,rating
0,190,7,2.0
1,325,7,3.5
2,493,7,3.0
3,533,7,2.5
4,783,7,3.5


In [3]:
df_train,df_test = train_test_split(df, test_size = TEST_SIZE, random_state=30)
print(df_train.shape,df_test.shape)
df_test.head(5)

(583, 3) (57798, 3)


,userID,movieID,rating
55282,37419,8970,3.0
44021,24495,8798,4.5
29144,67042,8361,3.5
8109,43779,858,5.0
20892,15061,965,4.0


In [4]:
user_test = df_test["userID"].tolist()
print(len(user_test))
user_test = list(set(user_test))
user_test.sort()
print(len(user_test))

57798
2071


In [5]:
file = open(DATAPATH + 'movieID.txt', 'r')
movieID = [int(x) for x in file]
file.close()
print(movieID)

file = open(DATAPATH + 'userID.txt', 'r')
userID = [int(x) for x in file]
file.close()
print(userID)

[7, 8, 9, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 784, 785, 786, 788, 789, 792, 795, 796, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 813, 814, 815, 818, 820, 821, 823, 824, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 858, 860, 861, 862, 864, 865, 866, 867, 868, 869, 870, 875, 876, 877, 879, 880, 881, 882, 884, 885, 886, 887, 888, 891, 892, 893, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 976, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 996, 997, 998, 999, 7843, 7844, 7845

In [6]:
movieOrder = [i for i in range(len(movieID))]
movie_dic = dict(zip(movieID, movieOrder))
print(movie_dic)

userOrder = [i for i in range(len(userID))]
user_dic = dict(zip(userID, userOrder))
print(user_dic)

{7: 0, 8: 1, 9: 2, 78: 3, 79: 4, 80: 5, 81: 6, 82: 7, 83: 8, 84: 9, 85: 10, 86: 11, 87: 12, 88: 13, 89: 14, 90: 15, 92: 16, 93: 17, 94: 18, 95: 19, 96: 20, 97: 21, 98: 22, 99: 23, 784: 24, 785: 25, 786: 26, 788: 27, 789: 28, 792: 29, 795: 30, 796: 31, 798: 32, 799: 33, 800: 34, 801: 35, 802: 36, 803: 37, 804: 38, 805: 39, 806: 40, 807: 41, 808: 42, 809: 43, 810: 44, 813: 45, 814: 46, 815: 47, 818: 48, 820: 49, 821: 50, 823: 51, 824: 52, 827: 53, 828: 54, 829: 55, 830: 56, 831: 57, 832: 58, 833: 59, 834: 60, 835: 61, 836: 62, 837: 63, 838: 64, 839: 65, 840: 66, 841: 67, 842: 68, 844: 69, 845: 70, 846: 71, 847: 72, 848: 73, 849: 74, 850: 75, 851: 76, 852: 77, 853: 78, 858: 79, 860: 80, 861: 81, 862: 82, 864: 83, 865: 84, 866: 85, 867: 86, 868: 87, 869: 88, 870: 89, 875: 90, 876: 91, 877: 92, 879: 93, 880: 94, 881: 95, 882: 96, 884: 97, 885: 98, 886: 99, 887: 100, 888: 101, 891: 102, 892: 103, 893: 104, 896: 105, 897: 106, 898: 107, 899: 108, 900: 109, 901: 110, 902: 111, 903: 112, 904: 1

In [7]:
FI_matrix = np.load(DATAPATH + "FI_matrix.npy")
print(FI_matrix.shape)

feature = np.load(DATAPATH + "features/low_feature.npy")
print(feature.shape)

(2072, 100)
(948, 100)


In [8]:
# user_test = [783]
# user_test = [783, 922]
hits = 0
for uid in user_test:
    tmp = df_train[df_train.userID == uid]["movieID"].tolist()
    # print(tmp)
    candidate = set(movieID) - set(tmp)
    # print(len(candidate), len(movieID))
    FI = FI_matrix[user_dic.get(uid)] # 用户喜好特征
    scores = {}
    for mid in candidate:
        fea = feature[movie_dic.get(mid)] # 候选视频特征
        score = (fea * FI).sum() # 1.喜好度视为权重
        # score = pearsonr(fea, FI)[0] # 2.皮尔逊相关系数
        scores[mid] = score
    sort_scores = sorted(scores.items(), key=lambda obj: obj[1], reverse=True)
    # print(912, scores.get(912))
    # print(sort_scores[:K])
    topk_list = [i for i,j in sort_scores[:K]]
    actual_list = df_test[df_test.userID == uid]["movieID"].tolist()
    hits += len(set(topk_list) & set(actual_list))
    # print(actual_list)
    # print(topk_list)
print(hits)

1539


In [9]:
actual_total = len(df_test)
precision = hits/(len(user_test)*K)
recall = hits/actual_total
print(hits, (len(user_test)*K), actual_total)
print(precision, recall)

1539 10355 57798
0.14862385321100918 0.026627218934911243


In [10]:
from scipy.stats import pearsonr
x = np.array([1,2,3])
y = np.array([1,2,4])
pccs = pearsonr(x, y)
pccs[0]

0.9819805060619659